In [3]:
!pip install torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 9.6 MB/s eta 0:00:00


In [4]:
# import PyTorch and iris dataset from scikit learn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import torch
from torcheval.metrics.functional import multiclass_accuracy

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [5]:
iris = load_iris()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

In [7]:
n_features, n_classes = X_train.shape[1], len(set(y_train))

In [8]:
X_pt_train = torch.FloatTensor(X_train)
y_pt_train = torch.LongTensor(y_train)
X_pt_test = torch.FloatTensor(X_test)
y_pt_test = torch.LongTensor(y_test)

In [17]:
class LogisticRegressionPT(torch.nn.Module):
    def __init__(self):
        super(LogisticRegressionPT, self).__init__()
        self.layer_1 = torch.nn.Linear(n_features, 10)
        self.layer_2 = torch.nn.Linear(10, 10)
        self.layer_3 = torch.nn.Linear(10, n_classes)

    def forward(self, x):
        x = torch.nn.functional.relu(self.layer_1(x))
        x = torch.nn.functional.relu(self.layer_2(x))
        x = torch.nn.functional.softmax(self.layer_3(x), dim=1)
        return x

In [18]:
model_pt = LogisticRegressionPT()

In [19]:
print(model_pt)

LogisticRegressionPT(
  (layer_1): Linear(in_features=4, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=3, bias=True)
)


In [20]:
learning_rate = 0.01
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_pt.parameters(), lr=learning_rate)

## Training

In [21]:
def train(model, optimizer, criterion, X, y, num_epochs, train_losses):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output_train = model(X)

        loss_train = criterion(output_train, y)
        loss_train.backward()
        optimizer.step()

        train_losses[epoch] = loss_train.item()

        if (epoch + 1) % 50 == 0:
            print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss_train.item():.4f}')

In [22]:
num_epochs = 100
train_losses = np.zeros(num_epochs)


In [23]:
train(model_pt, optimizer, criterion, X_pt_train, y_pt_train, num_epochs, train_losses)

Epoch: 50/100, Loss: 0.6065
Epoch: 100/100, Loss: 0.5777


In [24]:
predictions = torch.argmax(model_pt(X_pt_test), dim=1)
multiclass_accuracy(predictions, y_pt_test)

tensor(0.9667)